In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

import os
from tqdm import tqdm_notebook

# 작업파일 불러오기

- 전체 :  ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시', '갤럭시케이스',
          '안경','모자','하이힐','후드','아이폰케이스','아이폰','자켓','청바지','점퍼','중고차','키보드','노트북','마우스','목도리',
          '네일','니트','목걸이','나이키','닌텐도','원피스','패딩','바지','향수','냉장고','레고','반지','셔츠','치마','스피커','정장',
          '선글라스','티셔츠','트레이닝','티비','지갑','세탁기','시계']
- '아디다스 0','에어컨 1','에어팟 2','오토바이 3','블라우스 4','팔찌 5','카메라 6','코트7','컴퓨터 8','구두 9','귀걸이10','갤럭시 11'

In [3]:
#내 작업
query_list = ['아디다스','에어컨','에어팟','오토바이','블라우스','팔찌','카메라','코트','컴퓨터','구두','귀걸이','갤럭시']

In [5]:
total_df = []
for i in range(len(query_list)):
    total_df.append(pd.read_csv(f'/Users/ppangppang/Desktop/ssac/Final_project/2.Processed/info/{query_list[i]}.csv'))

# 태그 추천 테스트
1. 문장-단어 TF-IDF 구하기 : 한 문장(product)에 중요도가 높은 단어들 테이블
2. 문장-문장 유사도 구하기 : 특정 문장(prodcut)기준 문장간 유사도를 통해 특정 문장과 유사한 문장 뽑기
3. 1을 미리 구해놓음 -> 2에서 비슷한 문장 뽑음 -> 2에서 뽑힌 문장을 1에서 찾음 -> 1에서 태그 뽑기


- 구현에 필요한 내용
 - 1,2번 구현 테스트
 - 언어 전처리 필요

## 문단-단어 TF-IDF 구하기

In [46]:
adidas = total_df[0].copy()
adidas.fillna(0,inplace=True)
adidas.replace(0,'',inplace=True)
adidas['merge'] = adidas['title'] + adidas['keyword'] + adidas['cat1'] + adidas['cat2'] + adidas['cat3']
adidas

,product_id,title,keyword,cat1,cat2,cat3,view,image,merge
0,162427299,아디다스 문디알 풋살화265m,"아디다스,문디알,코파문디알,아디다스문디알,풋살화",스포츠/레저,축구,축구 용품,641,https://media.bunjang.co.kr/product/162427299_...,"아디다스 문디알 풋살화265m아디다스,문디알,코파문디알,아디다스문디알,풋살화스포츠/..."
1,172688578,아디다스 반팔티,"아디다스,아디다스반팔,반팔",남성의류,티셔츠,반팔 티셔츠,12,https://media.bunjang.co.kr/product/172688578_...,"아디다스 반팔티아디다스,아디다스반팔,반팔남성의류티셔츠반팔 티셔츠"
2,174560842,아디다스 봄가을 트레이닝 1230,100,남성의류,바지,기타(바지),4,https://media.bunjang.co.kr/product/174560842_...,아디다스 봄가을 트레이닝 1230100남성의류바지기타(바지)
3,171688500,아디다스 아이바 글러브 레드,"복싱,복싱글러브,아이바",스포츠/레저,기타 구기 스포츠,0,101,https://media.bunjang.co.kr/product/171688500_...,"아디다스 아이바 글러브 레드 복싱,복싱글러브,아이바스포츠/레저기타 구기 스포츠0"
4,174560837,아디다스 빅로고 버건디 후드티,"아디다스빅로고,아디다스후드티,빅로고후드,버건디후드",여성의류,후드티/후드집업,0,1,https://media.bunjang.co.kr/product/174560837_...,"아디다스 빅로고 버건디 후드티아디다스빅로고,아디다스후드티,빅로고후드,버건디후드여성의..."
...,...,...,...,...,...,...,...,...,...
5995,170889615,아디다스 x 고스티드+ 265 FG 최상급 축구화,"축구화,축구,아디다스x고스티드+,x고스티드,엑스",스포츠/레저,축구,축구 용품,390,https://media.bunjang.co.kr/product/170889615_...,"아디다스 x 고스티드+ 265 FG 최상급 축구화축구화,축구,아디다스x고스티드+,x..."
5996,118288417,아디다스 유로파 핑크 하의 85,"아디다스 유로파,유로파 85,유로파 핑크",여성의류,트레이닝,하의,291,https://media.bunjang.co.kr/product/118288417_...,"아디다스 유로파 핑크 하의 85아디다스 유로파,유로파 85,유로파 핑크여성의류트레이닝하의"
5997,118288210,아디다스 유로파 핑크 상의 85,"아디다스 유로파,85,유로파 핑크,아디다스 여자 상",여성의류,트레이닝,상의,152,https://media.bunjang.co.kr/product/118288210_...,"아디다스 유로파 핑크 상의 85아디다스 유로파,85,유로파 핑크,아디다스 여자 상여..."
5998,174460778,<240>아디다스 오즈위고 스웨이드 화이트 새상품,"아디다스,오즈위고,240,EG0546,화이트",신발,스니커즈,0,9,https://media.bunjang.co.kr/product/174460778_...,"<240>아디다스 오즈위고 스웨이드 화이트 새상품아디다스,오즈위고,240,EG054..."


In [49]:
adidas.iloc[[225]]

,product_id,title,keyword,cat1,cat2,cat3,view,image,merge
225,128398927,아디다스 엑스 19+ 핑크팩,,스포츠/레저,축구,축구 용품,2541,https://media.bunjang.co.kr/product/128398927_...,아디다스 엑스 19+ 핑크팩스포츠/레저축구축구 용품


In [47]:
test = adidas['merge'].to_list()

In [48]:
test

['아디다스 문디알 풋살화265m아디다스,문디알,코파문디알,아디다스문디알,풋살화스포츠/레저축구축구 용품',
 '아디다스 반팔티아디다스,아디다스반팔,반팔남성의류티셔츠반팔 티셔츠',
 '아디다스 봄가을 트레이닝 1230100남성의류바지기타(바지)',
 '아디다스 아이바 글러브 레드 복싱,복싱글러브,아이바스포츠/레저기타 구기 스포츠0',
 '아디다스 빅로고 버건디 후드티아디다스빅로고,아디다스후드티,빅로고후드,버건디후드여성의류후드티/후드집업0',
 '아디다스 쪼리아디다스,아디다스쪼리,슬리퍼,쪼리,아디다스슬리퍼신발남성화샌들/슬리퍼',
 '아디다스 니짜 운동화아디다스,니짜,아디다스스니커즈,운동화,아디다스운동화신발스니커즈0',
 '아디다스 스니커즈 240아디다스신발스니커즈0',
 '아디다스 커틀벨아디다스,커틀벨스포츠/레저헬스/요가/필라테스헬스/요가/필라테스 장비',
 '13FW라프시몬스X아디다스 테렉스 판매합니다.라프시몬스런닝화,라프시몬스운동화,라프시몬스260,라프시몬스,라프시몬스신발신발스니커즈0',
 '아디다스신발260아디다스,사이즈260신발스니커즈0',
 '아디다스 후리스팬츠  L아디다스팬츠,후리스남성의류트레이닝하의',
 '아디다스 운동화 255아디다스운동화,아디다스255,운동화255,adidas신발남성화기타(남성화)',
 '아디다스 스탠스미스 245아디다스245,스탠스미스245,아디다스,245신발스니커즈0',
 '팔라스 x 아디다스 트랙 팬츠 바지 us m 판매아디다스,파이어버드,트랙,트레이닝,팬츠남성의류트레이닝하의',
 '[90] 아디다스 유로파 남빨 져지 윤두준 져지나이키,뉴발,윤두준져지,윤두준,데상트여성의류트레이닝상의',
 '아디다스 키즈용 반바지아디다스,반바지,140,키즈용,스포츠유아동/출산여아의류(3-6세)팬츠',
 '[xl]아디다스 노바 윈드브레이커아디다스,바람막이,윈드브레이커,xl남성의류트레이닝상의',
 '아디다스 깔깔이 패딩 네이비 점퍼 90 95자켓,블루종,야구,네오,심플남성의류패딩/점퍼기타(패딩/점퍼)',
 '파워정품 아코탭 미시착새상품 아디다스오즈

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [53]:
countvect = vect.fit_transform(test)

In [54]:
countvect.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [55]:
vect.vocabulary_

{'아디다스': 6425,
 '문디알': 4391,
 '풋살화265m아디다스': 11340,
 '코파문디알': 10196,
 '아디다스문디알': 6847,
 '풋살화스포츠': 11345,
 '레저축구축구': 3892,
 '용품': 8655,
 '반팔티아디다스': 4740,
 '아디다스반팔': 6900,
 '반팔남성의류티셔츠반팔': 4694,
 '티셔츠': 10814,
 '봄가을': 4987,
 '트레이닝': 10647,
 '1230100남성의류바지기타': 152,
 '바지': 4569,
 '아이바': 7897,
 '글러브': 2651,
 '레드': 3840,
 '복싱': 4967,
 '복싱글러브': 4968,
 '아이바스포츠': 7898,
 '레저기타': 3886,
 '구기': 2544,
 '스포츠0': 6133,
 '빅로고': 5187,
 '버건디': 4813,
 '후드티아디다스빅로고': 11844,
 '아디다스후드티': 7763,
 '빅로고후드': 5193,
 '버건디후드여성의류후드티': 4816,
 '후드집업0': 11780,
 '쪼리아디다스': 9762,
 '아디다스쪼리': 7457,
 '슬리퍼': 6195,
 '쪼리': 9761,
 '아디다스슬리퍼신발남성화샌들': 7057,
 '니짜': 3368,
 '운동화아디다스': 8758,
 '아디다스스니커즈': 7038,
 '운동화': 8717,
 '아디다스운동화신발스니커즈0': 7263,
 '스니커즈': 5886,
 '240아디다스신발스니커즈0': 450,
 '커틀벨아디다스': 10122,
 '커틀벨스포츠': 10121,
 '레저헬스': 3894,
 '요가': 8636,
 '필라테스헬스': 11515,
 '필라테스': 11509,
 '장비': 9347,
 '13fw라프시몬스x아디다스': 169,
 '테렉스': 10495,
 '판매합니다': 10967,
 '라프시몬스런닝화': 3728,
 '라프시몬스운동화': 3730,
 '라프시몬스260': 3724,
 '라프시몬스': 3723,
 '라프시몬스신발신발스니커즈0

In [56]:
countvect_df = pd.DataFrame(countvect.toarray(), columns = sorted(vect.vocabulary_))
countvect_df.index = adidas['product_id']
countvect_df

,000아디다스,000아디다스유아동,001,001풋살화,018,01나이키,0204,032c,032c뮬,032c아디다스,032c아디다스신발스니커즈0,032c캠퍼스,06,067아디다스,07나이키,08,09,090아디다스,090여성의류트레이닝하의,095남성의류트레이닝상의,0k,0나이키,0아디다스,0이지부스트,10,100,100100,100100남성의류트레이닝상의,100105,1004026아디다스패션,10095,100c시계,100l,100l사이즈,100size아디다스유아동,100ㅡ105,100나이키,100남성의류맨투맨0,100남성의류반바지밴딩,100남성의류자켓바람막이,100남성의류트레이닝상의,100남성의류트레이닝세트,100남성의류트레이닝하의,100남성의류티셔츠반팔,100남성의류패딩,100남성의류후드티,100네파,100데상트,100사이즈,100사이즈100,100사이즈m사이즈,100사이즈긴팔티,100사이즈나이키,100사이즈남성의류맨투맨0,100사이즈남성의류조끼,100사이즈남성의류트레이닝상의,100사이즈사파리점퍼,100사이즈아디다스,100사이즈아디다스져지,100사이즈아디다스패딩,100수원삼성,100스포츠,100아디다스,100아디다스남성의류트레이닝상의,100아디다스남성의류트레이닝하의,100아디다스맨투맨,100아디다스올드스쿨,100아디다스져지,100아디다스패딩,100아디다스후드티,100여성의류맨투맨0,100여성의류자켓기타,100여성의류트레이닝상의,100올드스쿨,100입니다아디다스남성의류패딩,100폴로,100호아디다스져지,100후드,1020,10282,103,105,105105,105105남성의류트레이닝상의,105105남성의류후드티,105adidas,105xl남성의류후드티,105구스,105나이키,105남성의류맨투맨0,105남성의류반바지밴딩,105남성의류자켓기타,105남성의류자켓바람막이,105남성의류트레이닝상의,105남성의류트레이닝하의,105남성의류티셔츠반팔,105남성의류패딩,105남성의류후드티,105도,105반팔티셔츠남성의류티셔츠반팔,...,후디남성의류후드티,후디남여공용,후디드자켓,후디스포츠,후디아디다스,후디여성의류후드티,후디자켓100,후디집업,후루룩구제,후리수집업xxl남성의류트레이닝상의,후리스,후리스xl남성의류자켓기타,후리스고샤루브친스키,후리스남성의류자켓기타,후리스남성의류트레이닝하의,후리스남성의류패딩,후리스남성의류후드티,후리스빈티지,후리스쉐르파,후리스아디다스,후리스아디다스양털,후리스아디다스후리스,후리스양털후리스,후리스여성의류자켓기타,후리스여성의류패딩,후리스제니,후리스조거팬츠아디다스,후리스집업105남성의류후드티,후리스집업여성의류후드티,후리스팬츠,후리스푸마,후리스후드,후리스후드티트레이닝,후망,후망더비,후면,휠라,휠라남성의류트레이닝하의,휠라셔츠,휠라신발남성화기타,휠라신발스니커즈0,휠라여성의류패딩,휠라운동화,휠라운동화나이키,휠라키즈,휴먼,휴먼레이스,휴먼레이스아디다스,휴먼메이드,흑계아디다스,흑금,흑청,흑청바지,희귀,희귀남성의류트레이닝상의,희귀매물,희귀매물나이키,희귀사이즈,희귀색상여성의류트레이닝상의,희귀템,희기템,흰검나이키,흰검아디다스,흰검아디다스신발,흰녹,흰빨아디다스,흰색,흰색m,흰색롱패딩,흰색바람막이,흰색반팔,흰색비니,흰색아디다스,흰색아디다스티셔츠,흰색패션,흰아디다스,흰아디다스운동화,흰운동화,흰주,흰티여성의류티셔츠반팔,흰파,흰파검,흰파빨,흰핑,히어로구스다운,히트기어언더웨어,히프나틱,히피,힐탑시계,힙색,힙색가방남성가방기타,힙색가방여성가방기타,힙색나이키,힙색보조가방아디다스가방남성가방기타,힙색아디다스,힙색아디다스가방여성가방기타,힙색힙백가방여성가방기타,힙쌕가방남성가방기타,힙합,힙합신발스니커즈0
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
162427299,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
172688578,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
174560842,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
171688500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
174560837,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(countvect_df, countvect_df)

array([[1.        , 0.13483997, 0.13483997, ..., 0.12856487, 0.0805823 ,
        0.09534626],
       [0.13483997, 1.        , 0.2       , ..., 0.19069252, 0.11952286,
        0.14142136],
       [0.13483997, 0.2       , 1.        , ..., 0.19069252, 0.11952286,
        0.14142136],
       ...,
       [0.12856487, 0.19069252, 0.19069252, ..., 1.        , 0.11396058,
        0.13483997],
       [0.0805823 , 0.11952286, 0.11952286, ..., 0.11396058, 1.        ,
        0.08451543],
       [0.09534626, 0.14142136, 0.14142136, ..., 0.13483997, 0.08451543,
        1.        ]])

In [58]:
len(countvect_df.columns)

11968

### 필요한 컬럼만 보기

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features = 50)
tfvect = vect.fit(test)

In [60]:
tfidv_df = pd.DataFrame(tfvect.transform(test).toarray(), columns = sorted(vect.vocabulary_))
tfidv_df.index = adidas['product_id']
tfidv_df

,100,260,265,270,275,adidas,나이키,남성화,레저축구축구,맨투맨,바람막이,바지,반팔,블랙,새상품,슈퍼스타,스탠스미스,아디다스,아디다스져지,여성,여성화,오리지널,올드스쿨,용품,운동화,의류,이지,이지부스트,자켓,잡화,잡화신발,점퍼,점퍼기타,점퍼롱패딩,점퍼숏패딩,정품,져지,집업,축구화,출산유아동신발,트레이닝,트레이닝복,티셔츠,패딩,팬츠,프레데터,후드,후드집업,후드집업0,후드티
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
162427299,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.661334,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.221298,0.0,0.0,0.0,0.0,0.0,0.716704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
172688578,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.272095,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.96227,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
174560842,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.755896,0.0,0.0,0.0,0.0,0.0,0.172429,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.631577,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
171688500,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
174560837,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.285321,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.958432,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170889615,0.0,0.0,0.698408,0.0,0.0,0.0,0.0,0.0,0.473317,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.158383,0.0,0.0,0.0,0.0,0.0,0.512945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
118288417,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
118288210,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


## 문단-문단 비교하기

In [61]:
adidas

,product_id,title,keyword,cat1,cat2,cat3,view,image,merge
0,162427299,아디다스 문디알 풋살화265m,"아디다스,문디알,코파문디알,아디다스문디알,풋살화",스포츠/레저,축구,축구 용품,641,https://media.bunjang.co.kr/product/162427299_...,"아디다스 문디알 풋살화265m아디다스,문디알,코파문디알,아디다스문디알,풋살화스포츠/..."
1,172688578,아디다스 반팔티,"아디다스,아디다스반팔,반팔",남성의류,티셔츠,반팔 티셔츠,12,https://media.bunjang.co.kr/product/172688578_...,"아디다스 반팔티아디다스,아디다스반팔,반팔남성의류티셔츠반팔 티셔츠"
2,174560842,아디다스 봄가을 트레이닝 1230,100,남성의류,바지,기타(바지),4,https://media.bunjang.co.kr/product/174560842_...,아디다스 봄가을 트레이닝 1230100남성의류바지기타(바지)
3,171688500,아디다스 아이바 글러브 레드,"복싱,복싱글러브,아이바",스포츠/레저,기타 구기 스포츠,0,101,https://media.bunjang.co.kr/product/171688500_...,"아디다스 아이바 글러브 레드 복싱,복싱글러브,아이바스포츠/레저기타 구기 스포츠0"
4,174560837,아디다스 빅로고 버건디 후드티,"아디다스빅로고,아디다스후드티,빅로고후드,버건디후드",여성의류,후드티/후드집업,0,1,https://media.bunjang.co.kr/product/174560837_...,"아디다스 빅로고 버건디 후드티아디다스빅로고,아디다스후드티,빅로고후드,버건디후드여성의..."
...,...,...,...,...,...,...,...,...,...
5995,170889615,아디다스 x 고스티드+ 265 FG 최상급 축구화,"축구화,축구,아디다스x고스티드+,x고스티드,엑스",스포츠/레저,축구,축구 용품,390,https://media.bunjang.co.kr/product/170889615_...,"아디다스 x 고스티드+ 265 FG 최상급 축구화축구화,축구,아디다스x고스티드+,x..."
5996,118288417,아디다스 유로파 핑크 하의 85,"아디다스 유로파,유로파 85,유로파 핑크",여성의류,트레이닝,하의,291,https://media.bunjang.co.kr/product/118288417_...,"아디다스 유로파 핑크 하의 85아디다스 유로파,유로파 85,유로파 핑크여성의류트레이닝하의"
5997,118288210,아디다스 유로파 핑크 상의 85,"아디다스 유로파,85,유로파 핑크,아디다스 여자 상",여성의류,트레이닝,상의,152,https://media.bunjang.co.kr/product/118288210_...,"아디다스 유로파 핑크 상의 85아디다스 유로파,85,유로파 핑크,아디다스 여자 상여..."
5998,174460778,<240>아디다스 오즈위고 스웨이드 화이트 새상품,"아디다스,오즈위고,240,EG0546,화이트",신발,스니커즈,0,9,https://media.bunjang.co.kr/product/174460778_...,"<240>아디다스 오즈위고 스웨이드 화이트 새상품아디다스,오즈위고,240,EG054..."


In [73]:
adidas2 = adidas.copy()

In [78]:
tfidf = TfidfVectorizer(stop_words='english') # 한국어 불용어 전처리 필요

tfidf_matrix = tfidf.fit_transform(adidas2['merge'])
print(tfidf_matrix.shape)

(6000, 11954)


In [80]:
cosine_matrix = cosine_similarity(tfidf_matrix,tfidf_matrix)
cosine_matrix.shape

(6000, 6000)

In [81]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0043, 0.0038, ..., 0.004 , 0.0024, 0.0024],
       [0.0043, 1.    , 0.0074, ..., 0.0077, 0.0046, 0.0047],
       [0.0038, 0.0074, 1.    , ..., 0.0068, 0.004 , 0.0041],
       ...,
       [0.004 , 0.0077, 0.0068, ..., 1.    , 0.0042, 0.0043],
       [0.0024, 0.0046, 0.004 , ..., 0.0042, 1.    , 0.0025],
       [0.0024, 0.0047, 0.0041, ..., 0.0043, 0.0025, 1.    ]])

In [83]:
# 태그와 id를 매핑할 dict 생성
key2id = {}
for i,c in enumerate(adidas2['keyword']): key2id[i] = c

# id와 태그를 매핑할 dict 생성
id2key = {}
for i,c in key2id.items() : id2key[c] = i

In [92]:
idx = id2key['아디다스빅로고,아디다스후드티,빅로고후드,버건디후드']
sim_scores = [(i,c) for i,c in enumerate(cosine_matrix[idx]) if i != idx] # 자기자신 제외 상품들 유사도 및 인덱스 추출
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)

sim_scores[0:5]

[(4902, 0.28774613191860987),
 (2114, 0.2403121087948156),
 (5687, 0.23291099632960496),
 (4119, 0.2328902784894099),
 (4133, 0.2229765127093732)]

In [93]:
# 유사도 높은 아이디들의 태그 추천
sim_scores = [(key2id[i], score) for i, score in sim_scores[0:5]]
sim_scores

[('아디다스,아노락,버건디,105', 0.28774613191860987),
 ('아디다스,아디다스후드,adidas,아디다스후드티,블랙후드티', 0.2403121087948156),
 ('아디다스,아디다스후드티,후드티', 0.23291099632960496),
 ('아디다스후드티,아디다스,구제,빈티지,만원샵', 0.2328902784894099),
 ('아디다스,아디다스후드티,아디다스후드', 0.2229765127093732)]

# Konlpy 시도

In [3]:
from konlpy.tag import Okt

In [2]:
okt = Okt()
text = "아 나는 역시 밤에 일이 잘된다."

print(okt.morphs(text, stem=True))

['아', '나', '는', '역시', '밤', '에', '일이', '잘', '되다', '.']
